# Cut-in detection

In this notebook, the cut ins are detected. 

In [ ]:
import cutin_detection

In [ ]:
%debug

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp, kendalltau, spearmanr
from domain_model import StateVariable
from databaseemulator import DataBaseEmulator

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_cutin_scenarios", "database.json")
DBE = DataBaseEmulator(filename)
print("Number of scenarios: {:d}".format(len(DBE.collections["scenario"])))

In [ ]:
def cutin_parameters(scenario):
    # Obtain the duration of the lane change.
    activity = next(activity for activity in scenario.activities 
                    if activity.activity_category.name == 'lane change target')
    duration = activity.tduration
    ystart = activity.get_state(time=scenario.time["start"])[0]
    yend = activity.get_state(time=scenario.time["end"])[0]
    tstart = activity.tstart
    from_direction = 'r' if activity.name == "left lane change" else 'r'
    
    # Obtain longitudinal speed/position at time of start cut in.
    vstart = None
    xstart = None
    init_activity_target = ''
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LON_TARGET and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vstart = activity.get_state(time=scenario.time["start"])[0][0]
            xstart = activity.get_state(time=scenario.time["start"])[1][0]
            if activity.activity_category.name == "deceleration target":
                init_activity_target = 'd'
            elif activity.activity_category.name == "acceleration target":
                init_activity_target = 'a'
            else:
                init_activity_target = 'c'
            tstartlon = activity.tstart
            tendlin = activity.tend
            break
            
    # Obtain ego vehicle speed at time of start cut in.
    vego = None
    for actor, activity, _ in scenario.acts:
        if actor.name == "ego vehicle" and \
            activity.activity_category.state == StateVariable.SPEED and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vego = activity.get_state(time=scenario.time["start"])[0]
            
    return [duration, ystart, yend, xstart, vstart, vego,
            from_direction, init_activity_target, tstartlon, tendlin]

In [ ]:
# Get parameters of the cut-in: [duration, ystart, yend, xstart]
nscenarios = len(DBE.collections["scenario"])
#pars = [cutin_parameters(DBE.get_item("scenario", i)) for i in range(nscenarios)]
df = pd.DataFrame([cutin_parameters(DBE.get_item("scenario", i)) for i in range(nscenarios)], 
                  columns=['duration', 'ystart', 'yend', 'xstart', 'vstart', 'vego',
                           'from_direction', 'init_activity_target', 'tstartlon', 'tendlin'])
df["vdiff"] = df["vstart"] - df["vego"]
df["ystartabs"] = np.abs(df["ystart"])

# Plot a cut-in scenario

In [ ]:
def plot_cutin(scenario, axes=None):
    if axes is None:
        _, axes = plt.subplots(1, 1)
        axes.plot(0, 0, 'gx')
    df = pd.DataFrame(index=np.arange(scenario.time["start"], scenario.time["end"], 0.01),
                      columns=["x", "y", "act"])
    activity = next(activity for activity in scenario.activities 
                    if activity.activity_category.name == 'lane change target')
    df["y"] = activity.get_state(time=df.index.values)
    
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LON_TARGET:
            if activity.tstart < df.index[0]:
                i = 0
            else:
                i = df.index.get_loc(activity.tstart, method='ffill')
            if activity.tend > df.index[-1]:
                j = len(df)
            else:
                j = df.index.get_loc(activity.tend, method='bfill')
            df.loc[df.index[i]:df.index[j-1], "x"] = \
                activity.get_state(time=np.array(df.index[i:j]))[1]
            if activity.activity_category.name == "acceleration target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "a"
            elif activity.activity_category.name == "deceleration target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "d"
            elif activity.activity_category.name == "cruising target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "c"
            else:
                raise ValueError("Unknown longitudinal activity")
    for act, color in zip(["a", "d", "c"], ["g", "r", "b"]):
        mask = df["act"] == act
        if np.sum(mask):
            axes.plot(df.loc[mask, "x"], df.loc[mask, "y"], color=color)
    xmax = axes.get_xlim()[1]
    axes.set_xlim([-10, xmax])
    return axes

In [ ]:
axes = plot_cutin(DBE.get_item("scenario", 0))
for i in range(1, 20):
    plot_cutin(DBE.get_item("scenario", i), axes=axes)

# Step 1: Initial tags

In [ ]:
for activity, abbr in zip(["acceleration", "deceleration", "cruising"], ["a", "d", "c"]):
    print("Probability of {:12s}: {:4.1f} %".
          format(activity, np.sum(df["init_activity_target"] == abbr)/nscenarios*100))

Furthermore, the initial lateral activity is always a lane change.

# Step 2: Initial parameters

The initial parameters consists of:

- `ystart`: Initial lateral position w.r.t. center lane of ego
- `xstart`: Initial longitudinal offset w.r.t. ego vehicle
- `tlongitudinal`: Time at which the longitudinal activity starts

In [ ]:
# Determine if the parameters are correlated.
print("Variable 1  Variable 2  Correlation   p-value")
pars = ("ystart", "xstart", "tstartlon")
for i, par1 in enumerate(pars):
    for par2 in pars[i+1:]:
        kendall = kendalltau(df[par1], df[par2])
        print("{:>10s}  {:>10s}  {:11.3f}  {:8.2e}".format(par1, par2, kendall.correlation,
                                                           kendall.pvalue),
              "  {:s}".format("Correlated" if kendall.pvalue < 0.05 else ""))

In [ ]:
n = 100
spearmanr(np.linspace(0, 1, n), np.abs(np.linspace(-1, 1, n)))

In [ ]:
df['ystart']

In [ ]:
def get_ks_result(act1, act2, signal):
    values1 = df.loc[df["init_activity_target"] == act1, signal].values
    values2 = df.loc[df["init_activity_target"] == act2, signal].values
    result = ks_2samp(values1, values2)
    return result.pvalue

print("  Variable  KS test")
print("              a-d    a-c    d-c")
for signal in ['duration', 'ystart', 'yend', 'ystartabs']:
    print("{:>10s}".format(signal), end="")
    for act1, act2 in zip(["a", "a", "d"], ["d", "c", "c"]):
        print("  {:5.3f}".format(get_ks_result(act1, act2, signal)), end="")
    print("")

In [ ]:
for act in ['a', 'd', 'c']:
    n = np.sum(df["init_activity_target"] == act)
    plt.plot(np.sort(df.loc[df["init_activity_target"] == act, "ystart"]),
             np.linspace(0, 1, n), label=act)
plt.legend()

In [ ]:
# See if the lane change data is correlated.
print(spearmanr(df["ystart"], df["duration"]))
print(kendalltau(df["ystart"], df["duration"]))
print(spearmanr(df["yend"], df["duration"]))
print(kendalltau(df["yend"], df["duration"]))
print(spearmanr(df["yend"], df["ystart"]))
print(kendalltau(df["yend"], df["ystart"]))